<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/ConvolutionalNN/%D0%A1%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D0%B5_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_%D0%B4%D0%BB%D1%8F_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сверточные нейронные сети для анализа текста

## Доставляем и импортируем библиотеки

In [1]:
!pip install -U tensorflow pandas xlrd #tensorflow-gpu
!pip install keras scikit-learn pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.0 kB/s 
     |████████████████████████████████| 96 kB 5.0 MB/s 
     |████████████████████████████████| 438 kB 50.1 MB/s 
     |████████████████████████████████| 1.6 MB 34.5 MB/s 
     |████████████████████████████████| 5.8 MB 33.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninsta

In [2]:
import pandas as pd
import numpy as np
import re
import warnings

from pymorphy2 import MorphAnalyzer

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
for val in ['stopwords', 'punkt']:
  nltk.download(val)

import tensorflow as tf

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.callbacks import TensorBoard, EarlyStopping
from keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from gensim.models import Word2Vec

from tqdm import tqdm

warnings.filterwarnings('ignore')

tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Дополнительные функции

In [23]:
def clean_text(text):
  """
    Функция для предобработки текста
  """
  # Заменим пунктуацию на пробелы.
  text = re.sub("[^\w\s]", " ", str(text))
  # Заменим спецсимволы на пробелы и числа.
  text = re.sub("[^a-zA-Zа-яёА-ЯЁ\_]", " ", text)
  # Получаем токены
  text = word_tokenize(text)
  # Удаляем стоп слова
  text = [w for w in text if w not in stopwordslist]
  # Нормализация текста
  text = [morpher.parse(word)[0].normal_form for word in text if word not in stopwordslist]
  return " ".join(text)

def text_to_sequence(text, maxlen, vocabulary):
  """
    Функция для получения последовательности
  """
  tokens = word_tokenize(text.lower())
  tokens_filtered = [word for word in tokens if word.isalnum()]
  result = [vocabulary[word] for word in tokens_filtered if word in vocabulary]
  padding = [0]*(maxlen-len(result))
  return padding + result[-maxlen:]

def fit_evalute_model(model, x_train, y_train, x_test, y_test):

  model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
  
  model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.1)
  
  score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
  print('Test score:', score[0])
  print('Test accuracy:', score[1])

## Подготовим данные

In [29]:
#@title Глобальные переменные
max_words = 20000
max_len = 150
num_classes = 5
embedding_dim = 300
epochs = 20
batch_size = 512
print_batch_n = 100

stopwordslist = stopwords.words("russian")
morpher = MorphAnalyzer()

path_model = "./model/model.txt"

In [25]:
!wget -O 'отзывы за лето.xls' -qq --no-check-certificate https://gbcdn.mrgcdn.ru/uploads/asset/2800837/attachment/b61f2b2ddfad7d8623540e93cea07b20.xls
!wget -O 'model.zip' -qq --no-check-certificate http://vectors.nlpl.eu/repository/20/220.zip
!unzip 'model.zip' -d './model/'

Archive:  model.zip
  inflating: ./model/meta.json       
  inflating: ./model/model.bin       
  inflating: ./model/model.txt       
  inflating: ./model/README          


In [6]:
df = pd.read_excel('отзывы за лето.xls')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [8]:
df['preprocess'] = df['Content'].progress_apply(lambda x: clean_text(x))

100%|██████████| 20659/20659 [00:39<00:00, 519.79it/s]


## 1. Учим conv сеть для классификации

In [9]:
train_corpus = " ".join(df['preprocess'])
train_tokens = word_tokenize(train_corpus)
train_tokens_filtered = [word for word in train_tokens if word.isalnum()]

In [10]:
dist = FreqDist(train_tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'очень',
 'удобно',
 'всё',
 'работать',
 'удобный',
 'спасибо',
 'отлично',
 'нравиться',
 'я']

In [11]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [12]:
df_train_seq = np.asarray([text_to_sequence(text, max_len, vocabulary) for text in df['preprocess']], dtype=np.int32)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(df_train_seq, df['Rating'], test_size=0.33, random_state=42)

In [14]:
le = LabelEncoder()
y_train = le.fit_transform(y_train) 
y_test = le.transform(y_test)
le.classes_

array([1, 2, 3, 4, 5])

## 2. Рассмотрим два варианта сеточек


### 2.1 Инициализировав tf.keras.layers.Embedding предобученными векторами взятых из https://rusvectores.org/ru/


In [28]:
embeddings_index = {}
with open(path_model) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 249334 word vectors.


In [30]:
embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [31]:
model = Sequential([
 Embedding(input_dim=max_words, weights=[embedding_matrix], output_dim=embedding_dim, input_length=max_len),
 Conv1D(128, 3, activation="relu"),
 GlobalMaxPool1D(),
 Dense(10, activation="relu"),
 Dense(num_classes, activation="softmax"),
])

In [32]:
fit_evalute_model(model, x_train, y_train, x_test, y_test)

Epoch 1/20
25/25 [==============================] - 39s 2s/step - loss: 1.5978 - accuracy: 0.6775 - val_loss: 1.5847 - val_accuracy: 0.7011
Epoch 2/20
25/25 [==============================] - 40s 2s/step - loss: 1.5733 - accuracy: 0.7030 - val_loss: 1.5605 - val_accuracy: 0.7011
Epoch 3/20
25/25 [==============================] - 39s 2s/step - loss: 1.5492 - accuracy: 0.7030 - val_loss: 1.5370 - val_accuracy: 0.7011
Epoch 4/20
25/25 [==============================] - 39s 2s/step - loss: 1.5258 - accuracy: 0.7030 - val_loss: 1.5139 - val_accuracy: 0.7011
Epoch 5/20
25/25 [==============================] - 37s 1s/step - loss: 1.5030 - accuracy: 0.7030 - val_loss: 1.4916 - val_accuracy: 0.7011
Epoch 6/20
25/25 [==============================] - 36s 1s/step - loss: 1.4808 - accuracy: 0.7030 - val_loss: 1.4700 - val_accuracy: 0.7011
Epoch 7/20
25/25 [==============================] - 37s 1s/step - loss: 1.4593 - accuracy: 0.7030 - val_loss: 1.4488 - val_accuracy: 0.7011
Epoch 8/20
25/25 [==

### 2.2 Инициализировав слой tf.keras.layers.Embedding по умолчанию

In [20]:
model = Sequential([
 Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
 Conv1D(128, 3, activation="relu"),
 GlobalMaxPool1D(),
 Dense(10, activation="relu"),
 Dense(num_classes, activation="softmax"),
])

In [24]:
fit_evalute_model(model, x_train, y_train, x_test, y_test)

Epoch 1/20
25/25 [==============================] - 16s 624ms/step - loss: 0.3868 - accuracy: 0.8674 - val_loss: 0.8772 - val_accuracy: 0.7473
Epoch 2/20
25/25 [==============================] - 16s 625ms/step - loss: 0.3606 - accuracy: 0.8723 - val_loss: 0.9039 - val_accuracy: 0.7458
Epoch 3/20
25/25 [==============================] - 17s 662ms/step - loss: 0.3422 - accuracy: 0.8737 - val_loss: 0.9302 - val_accuracy: 0.7394
Epoch 4/20
25/25 [==============================] - 17s 673ms/step - loss: 0.3244 - accuracy: 0.8801 - val_loss: 0.9777 - val_accuracy: 0.7394
Epoch 5/20
25/25 [==============================] - 16s 634ms/step - loss: 0.3067 - accuracy: 0.8926 - val_loss: 1.0062 - val_accuracy: 0.7350
Epoch 6/20
25/25 [==============================] - 16s 621ms/step - loss: 0.2914 - accuracy: 0.9003 - val_loss: 1.0611 - val_accuracy: 0.7350
Epoch 7/20
25/25 [==============================] - 16s 630ms/step - loss: 0.2757 - accuracy: 0.9072 - val_loss: 1.1143 - val_accuracy: 0.7350

## 3. Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

**Вывод**:

В качестве предобученных весов использовалась сборка под индентификатором "ruwikiruscorpora_upos_cbow_300_10_2021". По результатам сетка, где использовался словарь именно из текста отработал лучше (*score*: 1.767443299293518, *accuracy*: 0.7329128980636597), чем сетка собранная из весов стороннего текста (*score*: 1.2222684621810913, *accuracy*: 0.7125256657600403).